In [1]:
import pandas as pd
import numpy as np
import requests
import yfinance as yf
from datetime import date
from datetime import datetime

In [2]:
tickers = ['XTN', 'HST']

In [3]:
asset_data = yf.download(tickers, start='2020-03-01', end='2021-03-01')

[*********************100%***********************]  2 of 2 completed


In [4]:
closes_df = asset_data['Close']

In [5]:
daily_returns = closes_df.pct_change()

In [6]:
covid_interest = pd.read_csv('covidKeywordInterest.csv', index_col='Week', parse_dates=True, infer_datetime_format=True)

In [7]:
twitter_sentiment = pd.read_csv('sentiment.csv', index_col='date', parse_dates=True, infer_datetime_format=True)

In [8]:
start = '2020-03-01'
end = '2021-03-01'

In [9]:
df = pd.DataFrame({'date':pd.date_range(start=start, end=end)})

In [10]:
weekly_scores = covid_interest['Interest Score'][8:62].values.tolist()

In [11]:
expanded_scores = []
for score in weekly_scores:
    for idx in range(0,7):
        expanded_scores.append(score)

In [12]:
ex_scs = expanded_scores[:(len(expanded_scores) - 12)]

In [13]:
df['Interest Score'] = ex_scs

In [14]:
df.set_index('date', inplace=True)

In [15]:
updated_df = pd.concat([df,daily_returns], axis=1, join='inner')

In [16]:
compound = twitter_sentiment['compound']

In [17]:
c_list = compound.values.tolist()

In [18]:
sentiments = []
for score in c_list:
    if score > .05:
        sentiments.append('positive')
    elif score < -.05:
        sentiments.append('negative')
    else:
        sentiments.append('neutral')

In [19]:
twitter_sentiment['sentiment'] = sentiments

In [20]:
#updated_df.head(30)

In [21]:
#twitter_sentiment.head(30)

In [22]:
master_df = pd.concat([updated_df,twitter_sentiment], axis=1, join='inner')

In [23]:
#master_df.head(30)

In [25]:
master_df.to_csv('master.csv')